# Summary

Apply Iain Marshall's CNN on our dataset. The CNN structure is defined in the https://onlinelibrary.wiley.com/doi/abs/10.1002/jrsm.1287

Pretrained embedding layer loaded from Iain.

Hyper parameters used:

Dropout: 0.160
Filter sizes: 1, 3, 5
Number of additional hidden layers: 0
Number of filters: 150
Maximum token features: 20000 rather than 12500
Number of epochs: rather than 20
Keep Normalization and drop for each Convolution layer.

Result: Precision 82%, recall 82% F1 81%

In [1]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from gensim.models import KeyedVectors
import pickle
from sklearn.model_selection import GridSearchCV, train_test_split
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.optimizers import Adam



MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 200

/home/dc/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 1 Vectorization

In [2]:
p_file = 'data_X'

with open(p_file, 'rb') as fin:
    data_X = pickle.load(fin)

In [3]:
p_file = 'RCT_labels'

with open(p_file, 'rb') as fin:
    labels = pickle.load(fin)

labels = to_categorical(np.asarray(labels))

In [4]:
p_file = 'Models/vectors'

with open(p_file, 'rb') as fin:
    data = pickle.load(fin)

In [5]:
p_file = 'Models/embedding_matrix'

with open(p_file, 'rb') as fin:
    embedding_matrix = pickle.load(fin)
    

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.25)

In [7]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(MAX_NUM_WORDS,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# 2 CNN by Iain Marshall

In [8]:
def model():
    # train a 1D convnet with global maxpooling
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    X = Conv1D(150, 1, activation='relu')(embedded_sequences)
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Dropout(0.16)(X)                                 # dropout
    X = MaxPooling1D(1)(X)

    X = Conv1D(150, 3, activation='relu')(X)
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Dropout(0.16)(X)                                 # dropout
    X = MaxPooling1D(3)(X)
    
    X = Conv1D(150, 5, activation='relu')(X)
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Dropout(0.16)(X)                                 # dropout
    X = MaxPooling1D(5)(X)

    X = GlobalMaxPooling1D()(X)
    preds = Dense(2, activation='softmax')(X)

    model = Model(inputs = sequence_input, outputs = preds)

        
    return model

In [9]:
model = model()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 200)         4000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 150)         30150     
_________________________________________________________________
batch_normalization_1 (Batch (None, 1000, 150)         600       
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 150)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1000, 150)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 998, 150)          67650     
__________

In [11]:
from keras.utils import plot_model
plot_model(model, to_file='cnnROBOT.png')

<img src='cnnROBOT.png'>

In [11]:
opt = Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [12]:
# Fit the model
model.fit(X_train, Y_train, batch_size = 64, epochs=20)

Epoch 1/20
14333/14333 [==============================] - 21s 1ms/step - loss: 0.6506 - acc: 0.7442
Epoch 2/20
14333/14333 [==============================] - 19s 1ms/step - loss: 0.4949 - acc: 0.7989
Epoch 3/20
14333/14333 [==============================] - 19s 1ms/step - loss: 0.4669 - acc: 0.8069
Epoch 4/20
14333/14333 [==============================] - 19s 1ms/step - loss: 0.4524 - acc: 0.8149
Epoch 5/20
14333/14333 [==============================] - 20s 1ms/step - loss: 0.4416 - acc: 0.8162
Epoch 6/20
14333/14333 [==============================] - 20s 1ms/step - loss: 0.4269 - acc: 0.8258
Epoch 7/20
14333/14333 [==============================] - 20s 1ms/step - loss: 0.4288 - acc: 0.8212
Epoch 8/20
14333/14333 [==============================] - 20s 1ms/step - loss: 0.4206 - acc: 0.8268
Epoch 9/20
14333/14333 [==============================] - 20s 1ms/step - loss: 0.4136 - acc: 0.8263
Epoch 10/20
14333/14333 [==============================] - 20s 1ms/step - loss: 0.4153 - acc: 0.8237

In [13]:
model_json = model.to_json()
with open ("Models/cnnROBOT.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("Models/cnnROBOT.h5")

In [14]:
loss, acc = model.evaluate(X_test, Y_test)
print("Validation set accuracy = ", acc)

4778/4778 [==============================] - 2s 418us/step
Validation set accuracy =  0.8156132272418546


In [15]:
from sklearn.metrics import classification_report

pred = model.predict(X_test, batch_size=32, verbose=1)
predicted = np.argmax(pred, axis=1)
report = classification_report(np.argmax(Y_test, axis=1), predicted)
print(report)

4778/4778 [==============================] - 2s 395us/step
             precision    recall  f1-score   support

          0       0.82      0.86      0.84      2630
          1       0.82      0.76      0.79      2148

avg / total       0.82      0.82      0.81      4778

